In [1]:
import numpy
import urllib
import scipy.optimize
import random
import math
from math import exp
from math import log

def parseData(fname):
  for l in urllib.urlopen(fname):
    yield eval(l)

print "Reading data..."
data = list(parseData("file:train.json"))
train_set = data[0:100000]
valid_set = data[100000:200000]
print "done"

Reading data...
done


In [2]:
users = []
items = []
users_items = {}
items_users = {}
datum_index = 0
for datum in train_set:
    u,i = datum['userID'],datum['businessID']
    if not u in users:
        users.append(u)
    if not i in items:
        items.append(i)
    if not users_items.has_key(u):
        users_items[u] = [(i,datum_index)]
    else:
        users_items[u].append((i,datum_index))
    if not items_users.has_key(i):
        items_users[i] = [(u,datum_index)]
    else:
        items_users[i].append((u,datum_index))
    datum_index += 1

In [3]:
sum_rating = 0
count_rating = 0
for u in users_items.keys():
    for i in users_items[u]:
        sum_rating += train_set[i[1]]['rating']
        count_rating += 1
alpha = float(sum_rating)/count_rating

In [4]:
print alpha

4.18703


In [5]:
beta_u = []
for u in users_items.keys():
    count_highrate = 0
    count_rate = 0
    for i in users_items[u]:
        count_rate += 1
        if train_set[i[1]]['rating'] > alpha:
            count_highrate += 1
        else:
            count_highrate -= 1
    beta_u.append(float(count_highrate)/count_rate)

In [6]:
beta_i = []
for i in items_users.keys():
    count_highrate = 0
    count_rate = 0
    for u in items_users[i]:
        count_rate += 1
        if train_set[u[1]]['rating'] > alpha:
            count_highrate += 1
        else:
            count_highrate -= 1
    beta_i.append(float(count_highrate)/count_rate)

In [7]:
def getIndex(a,b):
    count = 0
    for ele in b:
        if ele == a:
            return count
        count += 1

def f(Alpha,Betauser,Betaitem,lam):
    f_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            f_err += (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating'])**2 + \
                   lam * (Betauser[user_index]**2 + Betaitem[item_index]**2)            
    return f_err
                    
def fprime_user(Alpha,Betauser,Betaitem,lam):
    fprime_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            fprime_err += 2 * (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating']) + \
                       2 * lam * Betauser[user_index]
    return fprime_err

def fprime_item(Alpha,Betauser,Betaitem,lam):
    fprime_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            fprime_err += 2 * (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating']) + \
                       2 * lam * Betaitem[item_index]
    return fprime_err

def RMSE(Alpha,Betauser,Betaitem):
    square_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            square_err += (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating'])**2
    RMSE = math.sqrt(float(square_err)/len(train_set))
    return RMSE

def iterate(Alpha,Betauser,Betaitem,lam):
    # Update alpha
    sum_for_alpha = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            sum_for_alpha += train_set[i[1]]['rating'] - Betauser[user_index] - Betaitem[item_index]
    Alpha = float(sum_for_alpha) / len(train_set)
    # Update beta_user
    for u in users_items.keys():
        sum_for_betauser = 0
        count_item = 0
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            count_item += 1
            sum_for_betauser += train_set[i[1]]['rating'] - Alpha - Betaitem[item_index]
        Betauser[user_index] = float(sum_for_betauser) / (lam + count_item)
    # Update beta_item
    for i in items_users.keys():
        sum_for_betaitem = 0
        count_user = 0
        item_index = getIndex(i,items)
        for u in items_users[i]:
            user_index = getIndex(u[0],users)
            count_user += 1
            sum_for_betaitem += train_set[u[1]]['rating'] - Alpha - Betauser[user_index]
        Betaitem[item_index] = float(sum_for_betaitem) / (lam + count_user)
    return Alpha,Betauser,Betaitem

In [33]:
a = alpha
betau = beta_u
betai = beta_i
#print 'Initial RMSE = ' + str(RMSE(a,betau,betai))
print 'Iter\tRMSE'
for iter_time in range(10):
    count = iter_time + 1
    a,betau,betai = iterate(a,betau,betai,1.0)
    with open('lam-'+str(1)+'-'+str(count)+'-alpha.txt','w+') as f1:
        f1.write(str(a))
    f1.close()
    with open('lam-'+str(1)+'-'+str(count)+'-betauser.txt','w+') as f2:
        f2.write(str(betau))
    f2.close()
    with open('lam-'+str(1)+'-'+str(count)+'-betaitem.txt','w+') as f3:
        f3.write(str(betai))
    print str(count) + '\t' + str(RMSE(a,betau,betai))

Initial RMSE = 1.34444733684
Iter	RMSE

1	0.616840042629
2	0.578743047209
3	0.575713443033
4	0.57509578092
5	0.574873808236
6	0.574764790434
7	0.574701410554
8	0.574660863238
9	0.57463338643
10	0.574614061406


In [ ]:
Initial RMSE = 1.34444733684
1	0.616840042629
2	0.578743047209
3	0.575713443033
4	0.57509578092
5	0.574873808236
6	0.574764790434
7	0.574701410554
8	0.574660863238
9	0.57463338643
10	0.574614061406

In [35]:
with open('alpha.txt','w+') as f1:
    f1.write(str(a))
f1.close()
with open('beta_user.txt','w+') as f2:
    f2.write(str(betau))
f2.close()
with open('beta_item.txt','w+') as f3:
    f3.write(str(betai))
f3.close()

In [38]:
# MSE on validation set
valid_square_err = 0
for datum in valid_set:
    u = datum['userID']
    i = datum['businessID']
    if not u in users:
        valid_square_err += (a-datum['rating'])**2
        continue
    if not i in items:
        valid_square_err += (a-datum['rating'])**2
        continue
    u_index = getIndex(u,users)
    i_index = getIndex(i,items)
    valid_square_err += (a+betau[u_index]+betai[i_index]-datum['rating'])**2
valid_MSE = float(valid_square_err)/len(valid_set)

In [39]:
print valid_MSE

0.648616328111


In [40]:
average_beta_user = sum(betau)/len(betau)
average_beta_item = sum(betai)/len(betai)
# MSE on validation set
valid_square_err = 0
for datum in valid_set:
    u = datum['userID']
    i = datum['businessID']
    if not u in users:
        valid_square_err += (a+average_beta_user+average_beta_item-datum['rating'])**2
        continue
    if not i in items:
        valid_square_err += (a+average_beta_user+average_beta_item-datum['rating'])**2
        continue
    u_index = getIndex(u,users)
    i_index = getIndex(i,items)
    valid_square_err += (a+betau[u_index]+betai[i_index]-datum['rating'])**2
valid_MSE = float(valid_square_err)/len(valid_set)
print valid_MSE

0.648547133852


In [41]:
# Find max and min
print 'Users Min\t' + users[betau.index(min(betau))] + '\t' + str(min(betau))
print 'Users Max\t' + users[betau.index(max(betau))] + '\t' + str(max(betau))
print 'Items Min\t' + items[betai.index(min(betai))] + '\t' + str(min(betai))
print 'Items Max\t' + items[betai.index(max(betai))] + '\t' + str(max(betai))

Users Min	U417838537	-2.81039915248
Users Max	U357799541	1.17511838921
Items Min	B241777680	-2.25281230416
Items Max	B093985406	1.15363404482


In [28]:
def RMSE(Alpha,Betauser,Betaitem):
    square_err = 0
    for u in users_items.keys():
        user_index = getIndex(u,users)
        for i in users_items[u]:
            item_index = getIndex(i[0],items)
            square_err += (Alpha+Betauser[user_index]+Betaitem[item_index]-train_set[i[1]]['rating'])**2
    RMSE = math.sqrt(float(square_err)/len(train_set))
    return RMSE

In [42]:
RMSE(alpha,betau,betai)

0.5770364939576117

In [43]:
RMSE(alpha,beta_u,beta_i)

0.5770364939576117

In [14]:
def getIndex(a,b):
    count = 0
    for ele in b:
        if ele == a:
            return count
        count += 1